In [4]:
from scrapy.downloadermiddlewares.useragent import UserAgentMiddleware
from scrapy.downloadermiddlewares.retry import RetryMiddleware
from scrapy.utils.python import global_object_name
from fake_useragent import UserAgent
from scrapy import Item, Field, Selector, Request
from scrapy.spiders import Rule, Spider
from scrapy.linkextractors import LinkExtractor

In [2]:
class Items(Item):
    url = Field()
    name = Field()

In [5]:
class RotateUserAgentMiddleware(UserAgentMiddleware):
    def __init__(self, user_agent=''):
        self.user_agent = user_agent
    def process_request(self, request, spider):
        ua = UserAgent()
        request.headers.setdefault('User-Agent', ua.random)
        
class RotateIPMiddleware(RetryMiddleware):
    def __init__(self, ip=''):
        self.ip = ip
    def process_request(self, request, spider):
        request.meta["proxy"] = "http://" + self.ip

class MySpider(Spider):
    name = "leba"
    start_urls = ["https://www.al-akhbar.com/Editions"]
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
        'path.to.RotateUserAgentMiddleware': 400,
        'path.to.RotateIPMiddleware': 543,
        }
    }
    
    # rules = (
    #     Rule(LinkExtractor(restrict_css="container > l-content row archive-issues > l-main columns col-md-12 > editions-page section > archive-days-wrap"), follow=True),
    #     Rule(LinkExtractor(restrict_css=".product_pod > h3 > a"), callback="parse_book")
    # )

    def parse(self, response):
        
        selector = Selector(response)
        links = selector.css('.class day').xpath('@href').extract()
        
        for link in links:
            
            next_page_url = = selector.css('a.next-page').xpath('@href').extract_first()
            
            if next_page_url:
                yield Request(response.urljoin(next_page_url), callback=self.parse)



In [15]:
!scrapy

Scrapy 2.7.1 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  commands      
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command


In [ ]:
scrapy.Request(response.urljoin(next_page_url), callback=self.parse)

In [ ]:
# Follow the links to other pages
next_page_url = selector.css('a.next-page').xpath('@href').extract_first()
if next_page_url:
    yield scrapy.Request(response.urljoin(next_page_url), callback=self.parse)